In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import threading
import webbrowser
import os
import requests
import bs4
from urllib.request import urlopen
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import korean
import platform
from matplotlib import font_manager, rc
import matplotlib as mpl

In [11]:
'''
유튜브 랭킹 사이트(https://youtube-rank.com/board/bbs/board.php?bo_table=youtube)에서
카테고리, 제목, 구독자 등 최대 200개 이상 크롤링(selenium)하여
카테고리별 분석하여 그래프(파이 등)으로 시각화하기
'''

'\n유튜브 랭킹 사이트(https://youtube-rank.com/board/bbs/board.php?bo_table=youtube)에서\n카테고리, 제목, 구독자 등 최대 200개 이상 크롤링(selenium)하여\n카테고리별 분석하여 그래프(파이 등)으로 시각화하기\n'

In [ ]:
# -------------------------------------------
# 유튜브 채널 랭킹
# -------------------------------------------


# 유튜브 채널 랭킹 페이지 URL 설정
youtube_url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='


category_list = []
title_list = []
subscribes_list = []


# 기사를 크롤링하고 DataFrame에 저장하는 함수
def get_source(page) :
    
    # 크롬 웹 드라이버를 실행하여 브라우저를 여는 코드
    #driver = webdriver.Chrome("C:/Myexam/chromedriver/chromedriver.exe")
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    # 주어진 페이지 번호에 해당하는 뉴스 섹션 페이지로 이동
    driver.get(youtube_url + str(page))
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 페이지에서 모든 기사 리스트를 찾음
    category = driver.find_elements(By.CSS_SELECTOR,'.aos-init.aos-animate .subject')
    
    title = driver.find_elements(By.CSS_SELECTOR, '.aos-init.aos-animate .subject')
    
    subscribes = driver.find_elements(By.CSS_SELECTOR, '.aos-init.aos-animate')
    
    
    
    # ---- 카테고리 가져오기 ----
    for tag in category:
        try:
            # 제목 태그 찾기
            category_elements = tag.find_elements(By.CSS_SELECTOR, '#list-skin .category')
            
            # 태그가 있으면 제목 텍스트 저장, 없으면 "정보 없음"
            if category_elements:
                temp = category_elements[0].text.strip()
            else:
                temp = "해당 정보 없음"
            category_list.append(temp)
        except:
            print("제목 가져오는 중 에러 발생!")
    
    
    # ---- 제목 가져오기 ----
    for tag in title:
        try:
            # 제목 태그 찾기
            title_elements = tag.find_elements(By.CSS_SELECTOR, '#list-skin > form > table > tbody > tr > td.subject > h1 > a')
            
            # 태그가 있으면 제목 텍스트 저장, 없으면 "정보 없음"
            if title_elements:
                temp = title_elements[0].text.strip()
            else:
                temp = "해당 정보 없음"
            title_list.append(temp)
        except:
            print("제목 가져오는 중 에러 발생!")
            
            
            
    # ---- 구독자 가져오기 ----
    for tag in subscribes:
        try:
            # 제목 태그 찾기
            subscribes_elements = tag.find_elements(By.CSS_SELECTOR, '.subscriber_cnt')
            
            # 태그가 있으면 제목 텍스트 저장, 없으면 "정보 없음"
            if subscribes_elements:
                temp = subscribes_elements[0].text.strip()
            else:
                temp = "해당 정보 없음"
            subscribes_list.append(temp)
        except:
            print("제목 가져오는 중 에러 발생!")
            
            
            
    print("end :", page)

    # 브라우저 닫기
    driver.quit()      
    
    

for page in range(1, 4):
    get_source(page)                                                                            
    

end : 1
end : 2
end : 3


In [16]:
len(category_list)


300

In [17]:
len(title_list)


300

In [18]:
len(subscribes_list)

300